### Please read this before evaluating this notebook

Code for hyper parameter tuning is commented out to reduce run time of this notebook.
I can run that code during evaluation to justify the hyper parameters being used for various models.

In [114]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


In [115]:
import matplotlib.pyplot as plt

import pandas as pd

import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

### Data processing

In [116]:
train_df = pd.read_csv("/kaggle/input/students-drop-out-prediction/train.csv")
test_df = pd.read_csv("/kaggle/input/students-drop-out-prediction/test.csv")

In [117]:
all_col_names = list(set(train_df.columns) - set(["label", "id"])) 
binary_col_names = ["v_1", "v_26", "v_11", "v_14", "v_30", "v_28", "v_9", "v_27"]
non_binary_col_names = list(set(all_col_names) - set(binary_col_names))

In [118]:
transformer = ColumnTransformer(
    [
        ("drop_id", "drop", ["id"]),
         ("scale_non_binary", StandardScaler(), non_binary_col_names)
    ],
    remainder="passthrough"
)
transform_train_df = transformer.fit_transform(train_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_train_df = pd.DataFrame(data=transform_train_df, columns=feature_names)
transform_train_df = transform_train_df.astype(dict([(col, "int") for col in binary_col_names]))

In [119]:
transform_train_df1 = transform_train_df.copy()
y_train_com = transform_train_df1.pop("label")
X_train_com = transform_train_df1
X_train, X_valid, y_train, y_valid = train_test_split(X_train_com, y_train_com, test_size=0.3, random_state=1)

In [120]:
transform_test_data = transformer.fit_transform(test_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
X_test = pd.DataFrame(data=transform_test_data, columns=feature_names)
X_test = X_test.astype(dict([(col, "int") for col in binary_col_names]))

### Dummy model

In [121]:
# Dummy Classifier (most frequent)
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(X_train, y_train)
dummy_model.fit(X_train, y_train)
print(classification_report(y_valid, dummy_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       375
         1.0       0.47      1.00      0.64       540
         2.0       0.00      0.00      0.00       224

    accuracy                           0.47      1139
   macro avg       0.16      0.33      0.21      1139
weighted avg       0.22      0.47      0.30      1139



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Logistic Regression using SGDClassifier

In [122]:
# Logisitc Regression, Lasso

# model = SGDClassifier(random_state=1, loss="log", penalty="l1")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [123]:
# Logisitc Regression, Ridge

# model = SGDClassifier(random_state=1, loss="log", penalty="l2")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [124]:
# Logisitc Regression, Elasticnet

# model = SGDClassifier(random_state=1, loss="log", penalty="elasticnet")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [125]:
# Polynomial features (degree=2, including interaction) + Logistic Regression + Elasticnet

# transform_pipeline_poly = ColumnTransformer(
#     [
#         ("drop_id", "drop", ["id"]),
#         ("scale_non_binary", StandardScaler(), non_binary_col_names),
#         ("poly_non_binary", PolynomialFeatures(2), non_binary_col_names)
#     ],
#     remainder="passthrough"
# )
# transform_train_data_poly = transform_pipeline_poly.fit_transform(train_df)
# feature_names = [name.split("__")[1] for name in transform_pipeline_poly.get_feature_names_out()]
# transform_train_df_poly = pd.DataFrame(data=transform_train_data_poly, columns=feature_names)
# 
# y_train_com_poly = transform_train_df_poly.pop("label")
# X_train_com_poly = transform_train_df_poly
# 
# X_train_poly, X_valid_poly, y_train_poly, y_valid_poly = train_test_split(X_train_com_poly, y_train_com_poly, test_size=0.3, random_state=1)
# 
# model = SGDClassifier(random_state=1, loss="log", penalty="elasticnet")
# model.fit(X_train_poly, y_train_poly)
# preds = model.predict(X_valid_poly)
# print(classification_report(y_valid_poly, preds))

In [126]:
# Logistic Regression + degree 1 + elastic net + hyper-parameter tuning

# param_grid = {
#     "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
#     "learning_rate": ["optimal", "invscaling", "adaptive"],
#     "eta0": [1, 10, 100],
#     "penalty": ["l1", "l2", "elasticnet"]
# }
# model = SGDClassifier(random_state=1, loss="log", penalty="elasticnet")
# search = GridSearchCV(model, param_grid=param_grid, scoring="f1_macro", refit=True, cv=5)
# # search.fit(X_train_com, y_train_com)

In [127]:
lr_model = SGDClassifier(
    random_state=1,
    loss="log",
    penalty="elasticnet",
    alpha=0.0001,
    eta0=1,
    learning_rate="optimal",
)
lr_model.fit(X_train, y_train)
print(classification_report(y_valid, lr_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.47      0.91      0.62       375
         1.0       0.81      0.56      0.66       540
         2.0       0.48      0.06      0.11       224

    accuracy                           0.58      1139
   macro avg       0.59      0.51      0.46      1139
weighted avg       0.63      0.58      0.54      1139



### Clustering using KNeighborsClassifier

In [128]:
# Weights = inverse of distance

# model = KNeighborsClassifier(weights="distance")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [129]:
# Metric = Manhattan distance
# Weight = inverse of distance

# model = KNeighborsClassifier(weights="distance", p=1)
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [130]:
# Metric = Manhattan distance
# Weight = inverse of distance
# Algorithm = BallTree

# model = KNeighborsClassifier(weights="distance", p=1, algorithm="ball_tree")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [131]:
### Seems like changing algo does not have any affect, it just affects time taken
### Now we will perform hyper parameter tuning for the `leaf_size` parameter

# param_grid = {
#     "leaf_size": [5, 10, 30, 50, 100, 200]
# }
# model = KNeighborsClassifier(weights="distance", p=1)
# search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring="f1_macro", refit=True)
# search.fit(X_train_com, y_train_com)

In [132]:
clus_model = KNeighborsClassifier(weights="distance", p=1)
clus_model.fit(X_train, y_train)
print(classification_report(y_valid, clus_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.63      0.42      0.50       375
         1.0       0.59      0.83      0.69       540
         2.0       0.36      0.21      0.26       224

    accuracy                           0.57      1139
   macro avg       0.53      0.49      0.49      1139
weighted avg       0.56      0.57      0.54      1139



### Support Vector Machine

In [133]:
# RBF kernel

# model = SVC(random_state=1)
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [134]:
# Linear kernel

# model = SVC(random_state=1, kernel="linear")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [135]:
# Poly kernel, degree = 2

# model = SVC(random_state=1, degree=2, kernel="poly")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [136]:
# Poly kernel, degree = 3

# model = SVC(random_state=1, degree=3, kernel="poly")
# model.fit(X_train, y_train)
# preds = model.predict(X_valid)
# print(classification_report(y_valid, preds))

In [137]:
# HPT for rbf kernel (which performed best)

# param_grid = {
#     "C": [0.01, 0.1, 1, 10, 100],
#     "gamma": ["scale", 0.01, 0.1],
# }
# model = SVC(random_state=1, kernel="rbf")
# search = GridSearchCV(model, param_grid=param_grid, cv=3, scoring="f1_macro", refit=True)
# search.fit(X_train_com, y_train_com)

In [138]:
svm_model = SVC(random_state=1, kernel="rbf", C=10, gamma=0.01)
svm_model.fit(X_train, y_train)
print(classification_report(y_valid, svm_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       375
         1.0       0.47      1.00      0.64       540
         2.0       0.00      0.00      0.00       224

    accuracy                           0.47      1139
   macro avg       0.16      0.33      0.21      1139
weighted avg       0.22      0.47      0.30      1139



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Decision Tree

In [139]:
# With PCA (it did not help)

# pca = PCA(random_state=1)
# reduced_transform_train_df = pca.fit_transform(transform_train_df[non_binary_col_names])

# print(pca.explained_variance_ratio_[:4])

# reduced_transform_train_df = pd.DataFrame(data = reduced_transform_train_df[:,:4], columns = ["PC1", "PC2", "PC3", "PC4"])
# for col in binary_col_names:
#     reduced_transform_train_df[col] = transform_train_df[col]
# reduced_X_train_com = reduced_transform_train_df.copy()
# reduced_y_train_com = y_train_com
# reduced_X_train, reduced_X_valid, reduced_y_train, reduced_y_valid = train_test_split(reduced_X_train_com, reduced_y_train_com, test_size=0.3, random_state=1)

# model = DecisionTreeClassifier(class_weight="balanced", random_state=1, max_depth=5, ccp_alpha=0.001)
# cross_val_score(model, reduced_X_train_com, reduced_y_train_com, cv=5, scoring="f1_macro").mean()

In [140]:
# Without PCA

# model = DecisionTreeClassifier(class_weight="balanced", random_state=1, ccp_alpha=0.001, max_depth=8)
# cross_val_score(model, X_train_com, y_train_com, cv=5, scoring="f1_macro").mean()

In [141]:
# Grid search without PCA

# param_grid = {
#     "ccp_alpha": [0.0001, 0.001, 0.01],
#     "max_depth": [5, 10, 15],
#     "min_samples_split": [5, 10, 15],
#     "min_samples_leaf": [5, 10, 15],
#     "max_features": ["auto", "log2", None]
# }
# model = DecisionTreeClassifier(class_weight="balanced", random_state=1)
# search = GridSearchCV(model, param_grid=param_grid, scoring="f1_macro", refit=True, cv=5)
# search.fit(X_train_com, y_train_com)

In [142]:
tree_model = DecisionTreeClassifier(
    class_weight="balanced",
    random_state=1,
    ccp_alpha=0.001,
    max_depth=10,
    max_features=None,
    min_samples_leaf=10,
    min_samples_split=5
)
tree_model.fit(X_train, y_train)
print(classification_report(y_valid, tree_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.69      0.62      0.66       375
         1.0       0.79      0.66      0.72       540
         2.0       0.33      0.52      0.41       224

    accuracy                           0.62      1139
   macro avg       0.61      0.60      0.60      1139
weighted avg       0.67      0.62      0.64      1139



### Bagging

In [143]:
# HPT

# param_grid = {
#     "min_samples_split": [2, 5, 10],
#     "max_samples": [0.5, 0.9],
#     "n_estimators": [100, 200],
#     "min_samples_leaf": [10, 20],
#     "max_depth": [10, 20],
#     "ccp_alpha": [0.0001, 0.001]
# }
# model = RandomForestClassifier(
#     max_features=None,
#     random_state=1,
#     class_weight="balanced",
#     min_samples_leaf=10,
#     n_estimators=100
# )
# search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3)
# search.fit(X_train_com, y_train_com)

In [144]:
bag_model = RandomForestClassifier(
    max_features=None,
    random_state=1,
    class_weight="balanced",
    n_estimators=100,
    max_samples=0.9,
    min_samples_split=2,
    min_samples_leaf=10,
    max_depth=10,
    ccp_alpha=0.0001
)
bag_model.fit(X_train, y_train)
print(classification_report(y_valid, bag_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.79      0.65      0.71       375
         1.0       0.78      0.84      0.81       540
         2.0       0.45      0.52      0.48       224

    accuracy                           0.71      1139
   macro avg       0.68      0.67      0.67      1139
weighted avg       0.72      0.71      0.71      1139



### Boosting

In [145]:
# Tuning of parameters related to DecisionTree

# param_grid = {
#     "min_child_weight": [0.5, 1, 2],
#     "max_depth": [5, 10, 15],
#     "subsample": [0.5, 0.75, 1],
# }
# model = XGBClassifier(random_state=1)
# search = GridSearchCV(
#     model,
#     param_grid,
#     scoring="f1_macro",
#     refit=True,
#     cv=3
# )
# search.fit(X_train_com, y_train_com)
# print(search.best_params_)
# print(search.best_score_)

In [146]:
# Tuning of parameters related to boosting

# param_grid = {
#      "lambda": [1, 3, 5],
#      "alpha": [0, 1, 3],
#      "eta": [0.01, 0.05, 0.1],
# }
# model = XGBClassifier(
#     random_state=1,
#     max_depth=5,
#     min_child_weight=0.5,
#     subsample=0.75
# )
# search = GridSearchCV(
#     model,
#     param_grid,
#     scoring="f1_macro",
#     refit=True,
#     cv=3
# )
# search.fit(X_train_com, y_train_com)
# print(search.best_params_)
# print(search.best_score_)

In [147]:
kwargs = {"lambda": 5}
xgb_model = XGBClassifier(
    random_state=1,
    max_depth=5,
    min_child_weight=0.5,
    subsample=0.75,
    alpha=1,
    eta=0.1,
    **kwargs
)
xgb_model.fit(X_train, y_train)
print(classification_report(y_valid, xgb_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.78      0.73      0.75       375
         1.0       0.76      0.93      0.84       540
         2.0       0.59      0.33      0.42       224

    accuracy                           0.75      1139
   macro avg       0.71      0.66      0.67      1139
weighted avg       0.73      0.75      0.73      1139



### Multi layer Perceptron

In [148]:
# HPT

# param_grid = {
#     "hidden_layer_sizes": [(10), (30), (10, 10)],
#     "max_iter": [500, 1000],
#     "solver": ["sgd", "adam"],
#     "activation": ["relu", "sigmoid"],
#     "alpha": [0.01, 0.1],
# }
# model = MLPClassifier(random_state=1, learning_rate="adaptive")
# search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3, verbose=1)
# search.fit(X_train_com, y_train_com)
# print(search.best_params_)
# print(search.best_score_)

In [149]:
neural_model = MLPClassifier(
    random_state=1,
    learning_rate="adaptive",
    hidden_layer_sizes=(40),
    max_iter=500,
    solver="sgd",
    activation="relu",
    alpha=0.1
)
neural_model.fit(X_train, y_train)
print(classification_report(y_valid, neural_model.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.67      0.78      0.72       375
         1.0       0.71      0.91      0.80       540
         2.0       0.08      0.00      0.01       224

    accuracy                           0.69      1139
   macro avg       0.49      0.56      0.51      1139
weighted avg       0.57      0.69      0.62      1139



### Comparision of evaluation criteria (Mean F1 score OR Macro F1 score) for various models
- XG Boost: 0.68
- Random Forest: 0.67
- MLP: 0.67
- SVM: 0.66
- Logistic Regression: 0.64
- Decision Tree: 0.60
- KNeighbors Clustering: 0.58
- Dummy Classifier: 0.21

### Submit test predictions on best model (XG Boost)
#### Do not forget to train on the entire training dataset

In [150]:
xgb_model.fit(X_train_com, y_train_com)

preds = [int(i) for i in xgb_model.predict(X_test)]
submission = pd.DataFrame({"id": test_df["id"].to_numpy(), "label": preds})
submission.to_csv("submission.csv", index=False)